# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_data = pd.read_csv(
    "Output/weather_data.csv")

# Visualize
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Manokwari,-0.8667,134.0833,82.42,71,100,1.12,ID,1619831565
1,Vaini,-21.2000,-175.2000,78.80,89,75,12.66,TO,1619831277
2,Cidreira,-30.1811,-50.2056,68.79,71,0,16.96,BR,1619831474
3,Opuwo,-18.0607,13.8400,68.90,37,57,5.12,NaN,1619831565
4,Mar del Plata,-38.0023,-57.5575,63.00,77,0,10.36,AR,1619831306


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#configure gmaps
gmaps.configure(api_key=g_key)

max_humidity = weather_data["Humidity"].max()
locations = weather_data[["Lat", "Lng"]].astype(float)
humidity = weather_data["Humidity"].astype(float)

In [5]:
# Create a Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
min_temp = weather_data.loc[weather_data['Max Temp']<80]
max_temp = min_temp.loc[min_temp['Max Temp']>70]
wind_speed = max_temp.loc[max_temp['Wind Speed']<10]
cloudiness_0 = wind_speed.loc[wind_speed['Cloudiness']==0]
weather_conditions = cloudiness_0.loc[cloudiness_0['Humidity']<70]
weather_conditions

# Convert to DataFrame
hotel_df = pd.DataFrame(weather_conditions)
hotel_df = weather_conditions_df.reset_index(drop=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Gunnedah,-30.9833,150.2500,72.00,59,0,3.00,AU,1619831572,Mackellar Pet Friendly Motel Gunnedah
1,Atuona,-9.8000,-139.0333,79.18,69,0,2.24,PF,1619831573,Hôtel Hiva Oa Hanakee Lodge
2,Enshi,30.3000,109.4833,74.86,68,0,1.16,CN,1619831573,Moevenpick Enshi
3,Nanchong,30.7951,106.0847,79.45,54,0,3.02,CN,1619831578,Royal Oriental Garden Hotel
4,Teacapan,22.5500,-105.7500,76.73,56,0,7.00,MX,1619831627,Hotel los Mangos
5,Kiryat Gat,31.6100,34.7642,72.00,67,0,3.44,IL,1619831634,Desert Gat
6,Korla,41.7597,86.1469,71.96,16,0,3.42,CN,1619831435,The Garden Hotel Korla
7,Doha,25.2867,51.5333,78.01,69,0,4.61,QA,1619831642,The Curve Hotel
8,Quzhou,28.9594,118.8686,77.04,63,0,4.21,CN,1619831661,Quzhou International Hotel
9,Kunya,12.2233,8.5325,78.80,25,0,5.73,NG,1619831668,NaN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df['Hotel Name'] = ""

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Gunnedah,-30.9833,150.2500,72.00,59,0,3.00,AU,1619831572,
1,Atuona,-9.8000,-139.0333,79.18,69,0,2.24,PF,1619831573,
2,Enshi,30.3000,109.4833,74.86,68,0,1.16,CN,1619831573,
3,Nanchong,30.7951,106.0847,79.45,54,0,3.02,CN,1619831578,
4,Teacapan,22.5500,-105.7500,76.73,56,0,7.00,MX,1619831627,
5,Kiryat Gat,31.6100,34.7642,72.00,67,0,3.44,IL,1619831634,
6,Korla,41.7597,86.1469,71.96,16,0,3.42,CN,1619831435,
7,Doha,25.2867,51.5333,78.01,69,0,4.61,QA,1619831642,
8,Quzhou,28.9594,118.8686,77.04,63,0,4.21,CN,1619831661,
9,Kunya,12.2233,8.5325,78.80,25,0,5.73,NG,1619831668,


In [35]:
hotel_name = []
city_name = []
country_name = []
lat = []
lng = []

count=1

print('Beginning Data Retrieval')     
print('-----------------------------')

    # set up a parameters dictionary
params = {
"keyword": "Hotel",
"radius": 5000,
"type": "Hotel",
"key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_name.append(name_address['results'][0]['name'])
        city_name.append(hotel_df["City"])
        country_name.append(hotel_df["Country"])
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Beginning Data Retrieval
-----------------------------
Missing field/result... skipping.


In [36]:
weather_conditions_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Gunnedah,-30.9833,150.2500,72.00,59,0,3.00,AU,1619831572,Mackellar Pet Friendly Motel Gunnedah
1,Atuona,-9.8000,-139.0333,79.18,69,0,2.24,PF,1619831573,Hôtel Hiva Oa Hanakee Lodge
2,Enshi,30.3000,109.4833,74.86,68,0,1.16,CN,1619831573,Moevenpick Enshi
3,Nanchong,30.7951,106.0847,79.45,54,0,3.02,CN,1619831578,Royal Oriental Garden Hotel
4,Teacapan,22.5500,-105.7500,76.73,56,0,7.00,MX,1619831627,Hotel los Mangos
5,Kiryat Gat,31.6100,34.7642,72.00,67,0,3.44,IL,1619831634,Desert Gat
6,Korla,41.7597,86.1469,71.96,16,0,3.42,CN,1619831435,The Garden Hotel Korla
7,Doha,25.2867,51.5333,78.01,69,0,4.61,QA,1619831642,The Curve Hotel
8,Quzhou,28.9594,118.8686,77.04,63,0,4.21,CN,1619831661,Quzhou International Hotel
9,Kunya,12.2233,8.5325,78.80,25,0,5.73,NG,1619831668,NaN


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [43]:
fig=gmaps.figure()

# Assign the marker layer to a variable:
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map:
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [39]:
# Create a combined map:
fig=gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))